In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2


In [2]:
all_seasons = "all_seasons.csv"
all_seasons_df = pd.read_csv(all_seasons)
all_seasons_df.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Chris Robinson,VAN,23.0,195.58,90.71840,Western Kentucky,USA,1996,2,...,4.6,1.7,1.6,-11.4,0.039,0.088,0.155,0.486,0.156,1996-97
1,1,Matt Fish,MIA,27.0,210.82,106.59412,North Carolina-Wilmington,USA,1992,2,...,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97
2,2,Matt Bullard,HOU,30.0,208.28,106.59412,Iowa,USA,Undrafted,Undrafted,...,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97
3,3,Marty Conlon,BOS,29.0,210.82,111.13004,Providence,USA,Undrafted,Undrafted,...,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97
4,4,Martin Muursepp,DAL,22.0,205.74,106.59412,None,USA,1996,1,...,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97


In [3]:
# Create look up table for team_abbreviation
team = pd.DataFrame(all_seasons_df.team_abbreviation.unique())
team.columns = ['team_name']
team.insert(0, 'team_id', range(1, 1 + len(team)))
team
team.head(-10)

,team_id,team_name
0,1,VAN
1,2,MIA
2,3,HOU
3,4,BOS
4,5,DAL
5,6,TOR
6,7,CLE
7,8,GSW
8,9,IND
9,10,PHI


In [4]:
# Create look up table for team_abbreviation
players = pd.DataFrame(all_seasons_df.player_name.unique())
players.columns = ['players_name']
players.insert(0, 'players_id', range(1, 1 + len(players)))
players
players.head()

,players_id,players_name
0,1,Chris Robinson
1,2,Matt Fish
2,3,Matt Bullard
3,4,Marty Conlon
4,5,Martin Muursepp


In [5]:
# Create look up table for college
college = pd.DataFrame(all_seasons_df.college.unique())
college.columns = ['college_name']
college.insert(0, 'college_id', range(1, 1 + len(college)))
college

,college_id,college_name
0,1,Western Kentucky
1,2,North Carolina-Wilmington
2,3,Iowa
3,4,Providence
4,5,None
...,...,...
283,284,UNLV
284,285,Georgia State
285,286,USC
286,287,Ohio U.


In [6]:
# Create look up table for country
country = pd.DataFrame(all_seasons_df.country.unique())
country.columns = ['country_name']
country.insert(0, 'country_id', range(1, 1 + len(country)))
country

,country_id,country_name
0,1,USA
1,2,Jamaica
2,3,Canada
3,4,Croatia
4,5,Serbia and Montenegro
...,...,...
64,65,Cabo Verde
65,66,South Sudan
66,67,Bahamas
67,68,Ghana


In [7]:
# Split all_seasons_df in two
# Demographic and Stat.
demographic = all_seasons_df[['player_name', 'team_abbreviation', 'age', 'player_height','player_weight',
                             'college', 'country','draft_year','draft_round','draft_number']]
stat = all_seasons_df[['player_name', 'gp', 'pts', 'reb','ast','net_rating','oreb_pct','dreb_pct','usg_pct','ts_pct','ast_pct',
                      'season']]
print(demographic.head())
print(stat.head())

       player_name team_abbreviation   age  player_height  player_weight  \
0   Chris Robinson               VAN  23.0         195.58       90.71840   
1        Matt Fish               MIA  27.0         210.82      106.59412   
2     Matt Bullard               HOU  30.0         208.28      106.59412   
3     Marty Conlon               BOS  29.0         210.82      111.13004   
4  Martin Muursepp               DAL  22.0         205.74      106.59412   

                     college country draft_year draft_round draft_number  
0           Western Kentucky     USA       1996           2           51  
1  North Carolina-Wilmington     USA       1992           2           50  
2                       Iowa     USA  Undrafted   Undrafted    Undrafted  
3                 Providence     USA  Undrafted   Undrafted    Undrafted  
4                       None     USA       1996           1           25  
       player_name  gp  pts  reb  ast  net_rating  oreb_pct  dreb_pct  \
0   Chris Robinson  

In [8]:
# Put ids
demographic['players_id'] = demographic['player_name'].map(players.set_index('players_name')['players_id'])
demographic['team_id'] = demographic['team_abbreviation'].map(team.set_index('team_name')['team_id'])
demographic['college_id'] = demographic['college'].map(college.set_index('college_name')['college_id'])
demographic['country_id'] = demographic['country'].map(country.set_index('country_name')['country_id'])

C:\Users\Kyoo Ha Cha\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Kyoo Ha Cha\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Kyoo Ha Cha\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [9]:
demographic.head()

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,players_id,team_id,college_id,country_id
0,Chris Robinson,VAN,23.0,195.58,90.71840,Western Kentucky,USA,1996,2,51,1,1,1,1
1,Matt Fish,MIA,27.0,210.82,106.59412,North Carolina-Wilmington,USA,1992,2,50,2,2,2,1
2,Matt Bullard,HOU,30.0,208.28,106.59412,Iowa,USA,Undrafted,Undrafted,Undrafted,3,3,3,1
3,Marty Conlon,BOS,29.0,210.82,111.13004,Providence,USA,Undrafted,Undrafted,Undrafted,4,4,4,1
4,Martin Muursepp,DAL,22.0,205.74,106.59412,None,USA,1996,1,25,5,5,5,1


In [10]:
season78 = "season78.csv"
season78_df = pd.read_csv(season78)
season78_df.head()

,Season,Player,WS
0,1979,Aaron James,1.7
1,1979,Adrian Dantley*,5.7
2,1979,Al Skinner,1.2
3,1979,Alex English*,7.0
4,1979,Allan Bristow,3.5


In [16]:
season78_df["Player"]= season78_df["Player"].str.split("*", n = 1, expand = True)
season78_df.head()

,Season,Player,WS
0,1979,Aaron James,1.7
1,1979,Adrian Dantley,5.7
2,1979,Al Skinner,1.2
3,1979,Alex English,7.0
4,1979,Allan Bristow,3.5


In [17]:
season78_df

,Season,Player,WS
0,1979,Aaron James,1.7
1,1979,Adrian Dantley,5.7
2,1979,Al Skinner,1.2
3,1979,Alex English,7.0
4,1979,Allan Bristow,3.5
...,...,...,...
15308,2016,Willie Reed,1.2
15309,2016,Xavier Munford,0.1
15310,2016,Zach LaVine,2.6
15311,2016,Zach Randolph,4.3


In [18]:
#season78_df["name_year"] = season78_df["Player"] + " " + season78_df["Season"].map(str)
#season78_df.head()

In [19]:
season78_df['players_id'] = season78_df['Player'].map(players.set_index('players_name')['players_id'])
#season78_df = season78_df["players_id"].fillna(0, inplace = True)
#season78_df.Players_ID = season78_df.Players_ID.astype(int)
season78_df.head(-10)


,Season,Player,WS,players_id
0,1979,Aaron James,1.7,NaN
1,1979,Adrian Dantley,5.7,NaN
2,1979,Al Skinner,1.2,NaN
3,1979,Alex English,7.0,NaN
4,1979,Allan Bristow,3.5,NaN
...,...,...,...,...
15298,2016,Tyus Jones,0.3,1759.0
15299,2016,Udonis Haslem,0.4,904.0
15300,2016,Victor Oladipo,4.9,1626.0
15301,2016,Vince Carter,1.8,595.0


In [20]:
stat['players_id'] = stat['player_name'].map(players.set_index('players_name')['players_id'])
stat.head()

C:\Users\Kyoo Ha Cha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,player_name,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,players_id
0,Chris Robinson,41,4.6,1.7,1.6,-11.4,0.039,0.088,0.155,0.486,0.156,1996-97,1
1,Matt Fish,6,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97,2
2,Matt Bullard,71,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97,3
3,Marty Conlon,74,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97,4
4,Martin Muursepp,42,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97,5


In [21]:
# 3 Main Dataframes and 4 lookup tables
# 3 Mains are: stat, season78_df and demographic
# 4 lookups are: players, team, college, country

In [22]:
stat = stat.drop(columns=['player_name'])
stat.head()

,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,players_id
0,41,4.6,1.7,1.6,-11.4,0.039,0.088,0.155,0.486,0.156,1996-97,1
1,6,0.3,0.8,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97,2
2,71,4.5,1.6,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97,3
3,74,7.8,4.4,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97,4
4,42,3.7,1.6,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97,5


In [23]:
demographic = demographic.drop(columns=['player_name', 'team_abbreviation','college','country'])
demographic.head()

,age,player_height,player_weight,draft_year,draft_round,draft_number,players_id,team_id,college_id,country_id
0,23.0,195.58,90.71840,1996,2,51,1,1,1,1
1,27.0,210.82,106.59412,1992,2,50,2,2,2,1
2,30.0,208.28,106.59412,Undrafted,Undrafted,Undrafted,3,3,3,1
3,29.0,210.82,111.13004,Undrafted,Undrafted,Undrafted,4,4,4,1
4,22.0,205.74,106.59412,1996,1,25,5,5,5,1


In [24]:
season78_df = season78_df.drop(columns=['Player'])
season78_df.head(-10)

,Season,WS,players_id
0,1979,1.7,NaN
1,1979,5.7,NaN
2,1979,1.2,NaN
3,1979,7.0,NaN
4,1979,3.5,NaN
...,...,...,...
15298,2016,0.3,1759.0
15299,2016,0.4,904.0
15300,2016,4.9,1626.0
15301,2016,1.8,595.0


In [25]:
season78_df = season78_df.rename(columns={"Season": "season", "WS": "ws"})
season78_df.head()

,season,ws,players_id
0,1979,1.7,NaN
1,1979,5.7,NaN
2,1979,1.2,NaN
3,1979,7.0,NaN
4,1979,3.5,NaN


In [26]:
season78_df = season78_df.dropna(subset=['players_id'])

In [27]:
season78_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11071 entries, 30 to 15312
Data columns (total 3 columns):
season        11071 non-null int64
ws            11071 non-null float64
players_id    11071 non-null float64
dtypes: float64(2), int64(1)
memory usage: 346.0 KB


In [28]:
rds_connection_string = "postgres:postgres@localhost:5432/nba_db3"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [29]:
engine.table_names()

['players', 'stat', 'team', 'college', 'country', 'demographic', 'season78']

In [30]:
players.to_sql(name='players', con=engine, if_exists='replace', index=False)
season78_df.to_sql(name='season78', con=engine, if_exists='replace', index=False)
stat.to_sql(name='stat', con=engine, if_exists='replace', index=False)
demographic.to_sql(name='demographic', con=engine, if_exists='replace', index=False)
team.to_sql(name='team', con=engine, if_exists='replace', index=False)
college.to_sql(name='college', con=engine, if_exists='replace', index=False)
country.to_sql(name='country', con=engine, if_exists='replace', index=False)

In [31]:
#stat.to_sql(name='stat', con=engine, if_exists='append', index=False)
#demographic.to_sql(name='demographic', con=engine, if_exists='append', index=False)
#team.to_sql(name='team', con=engine, if_exists='append', index=False)
#college.to_sql(name='college', con=engine, if_exists='append', index=False)
#country.to_sql(name='country', con=engine, if_exists='append', index=False)

In [32]:
#demographic.to_sql(name='demographic',if_exists='replace', con=engine, index=False)

In [33]:
#season78_df.to_sql(name='season78', con=engine, if_exists='replace', index=False)

In [34]:
#stat.to_sql(name='stat', con=engine, if_exists='replace', index=False)

In [35]:
stat.to_sql(name='stat', con=engine, if_exists='append', index=False)

In [36]:
pd.read_sql_query('select * from season78', con=engine)

,season,ws,players_id
0,1979,7.7,1154.0
1,1979,6.3,434.0
2,1979,3.6,66.0
3,1979,3.4,824.0
4,1979,7.1,169.0
...,...,...,...
11066,2016,1.2,1755.0
11067,2016,0.1,1754.0
11068,2016,2.6,1672.0
11069,2016,4.3,795.0
